In [4]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.worksheet import Worksheet
import os
import datetime
import re


In [5]:
df = pd.read_excel('Pricing.xlsx',sheet_name="Sheet2")

#df.info()

df.columns

Index(['BRAND', 'PRODUCT SKU', 'PRODUCT NAME', 'Shop Name', 'PRODUCT LINK',
       'Note', 'BUNDLE LINK', 'Comment', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')

In [6]:
df['PRODUCT SKU'] = df['PRODUCT SKU'].str.strip().str.upper()
df['PRODUCT NAME'] = df['PRODUCT NAME'].str.strip().str.upper()
df['Shop Name'] = df['Shop Name'].str.strip().str.upper()

# Append ' EBAY' to "Shop Name" where it equals 'WA INDUSTRIAL SUPPLIES'
df.loc[df['Shop Name'] == 'WA INDUSTRIAL SUPPLIES', 'Shop Name'] += ' EBAY'


df_sub = df[['BRAND', 'PRODUCT SKU', 'PRODUCT NAME', 'Shop Name', 'PRODUCT LINK',
       'Note', 'BUNDLE LINK', 'Comment']].copy()

In [7]:
# Forward fill only select columns
df_sub[['BRAND', 'PRODUCT SKU','PRODUCT NAME']] = df_sub[['BRAND', 'PRODUCT SKU','PRODUCT NAME']].ffill()


df_sub.head()

,BRAND,PRODUCT SKU,PRODUCT NAME,Shop Name,PRODUCT LINK,Note,BUNDLE LINK,Comment
0,Unimig,U11005K,VIPER 135,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...,NaN,NaN,NaN
1,Unimig,U11005K,VIPER 135,ELECTROWELD EBAY,https://www.ebay.com.au/itm/275880137475?itmme...,NaN,NaN,NaN
2,Unimig,U11005K,VIPER 135,HAMPDON EBAY,https://www.ebay.com.au/sch/i.html?_from=R40&_...,NaN,NaN,NaN
3,Unimig,U11005K,VIPER 135,WA INDUSTRIAL SUPPLIES WEBSITE,https://www.waindustrialsupplies.net/product/v...,NaN,NaN,NaN
4,Unimig,U11005K,VIPER 135,WA INDUSTRIAL SUPPLIES EBAY,https://www.ebay.com.au/itm/186207493192?epid=...,NaN,NaN,NaN


In [8]:
#Example for Electroweld subset
elctro = df_sub[df_sub['Shop Name'].str.contains('electroweld', case=False, na=False, regex=True)][['Shop Name','PRODUCT LINK']]

elctro

,Shop Name,PRODUCT LINK
0,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...
1,ELECTROWELD EBAY,https://www.ebay.com.au/itm/275880137475?itmme...
22,ELECTROWELD EBAY,https://www.ebay.com.au/itm/285104945700?itmme...
25,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...
35,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...
...,...,...
1033,ELECTROWELD EBAY,NaN
1061,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...
1062,ELECTROWELD EBAY,https://www.ebay.com.au/itm/275214903410?_skw=...
1092,ELECTROWELD WEBSITE,NaN


In [9]:
import httpx
from parsel import Selector

def get_price_original(url: str) -> str:
    """
    Given an eBay product URL, fetches the page and returns the original price.
    """
    # First check if url is a valid, non-empty string.
    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan

    # Now safe to check length (though this is redundant with url.strip() check)
    if len(url) == 0:
        return -1
    
    # establish our HTTP2 client with browser-like headers
    session = httpx.Client(
        headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.35",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.7",
            "Accept-Language": "en-US,en;q=0.9",
            "Accept-Encoding": "gzip, deflate, br",
        },
        http2=True,
        follow_redirects=True
    )
    
    # fetch the webpage
    response = session.get(url)
    sel = Selector(response.text)
    
    # helper function to extract and strip text via CSS selectors
    css = lambda query: sel.css(query).get(default="").strip()
    
    # extract the original price
    price_original = css(".x-price-primary>span::text")
    
    return price_original

# Example usage:
url = "https://www.ebay.com.au/itm/275880137475?itmmeta=01J3RRGZ6RGRZ1PA0NB89D3ANK&hash=item403bbcd303:g:FG4AAOSwW1lmc0i2:sc:AU_RegularParcelWithTrackingAndSignature!2190!AU!-1&itmprp=enc%3AAQAJAAAA0Jo7zG6ZrYn%2F5GTELNdvHESu%2F8%2BPfiWY9kU--0LUo18ZDC%2BkIaIZvbRp1qI9So5TpISckKJtf4oGgV7V5XLYXMhG%2FLVX497kkF4F%2BcZyu0ELdaH4TYhi3PZ%2B%2BG7xxw75rLO07cwbga6GANQK6eH5xnlQBFG02TBJBi4CCfYH8LVydeSYVPiJ8rWQKbaKABcsxdxl%2B4uJjP6Ops8YFB%2BzE7T3l62OMSCzA63O1oEmgLiiOQySiIwlAiRx%2Fu3oj4L0GrlW2ZSPf1mHsjewL9BaDsQ%3D%7Ctkp%3ABk9SR7zzw5ieZA"
print(get_price_original(url))

AU $378.95


In [10]:
df_sub.head()

,BRAND,PRODUCT SKU,PRODUCT NAME,Shop Name,PRODUCT LINK,Note,BUNDLE LINK,Comment
0,Unimig,U11005K,VIPER 135,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...,NaN,NaN,NaN
1,Unimig,U11005K,VIPER 135,ELECTROWELD EBAY,https://www.ebay.com.au/itm/275880137475?itmme...,NaN,NaN,NaN
2,Unimig,U11005K,VIPER 135,HAMPDON EBAY,https://www.ebay.com.au/sch/i.html?_from=R40&_...,NaN,NaN,NaN
3,Unimig,U11005K,VIPER 135,WA INDUSTRIAL SUPPLIES WEBSITE,https://www.waindustrialsupplies.net/product/v...,NaN,NaN,NaN
4,Unimig,U11005K,VIPER 135,WA INDUSTRIAL SUPPLIES EBAY,https://www.ebay.com.au/itm/186207493192?epid=...,NaN,NaN,NaN


In [11]:
# Subset dataframe to retain only EBAY shop records
df_ebay = df_sub[df_sub['Shop Name'].str.contains('EBAY', case=False, na=False, regex=True)]

df_ebay

,BRAND,PRODUCT SKU,PRODUCT NAME,Shop Name,PRODUCT LINK,Note,BUNDLE LINK,Comment
1,Unimig,U11005K,VIPER 135,ELECTROWELD EBAY,https://www.ebay.com.au/itm/275880137475?itmme...,NaN,NaN,NaN
2,Unimig,U11005K,VIPER 135,HAMPDON EBAY,https://www.ebay.com.au/sch/i.html?_from=R40&_...,NaN,NaN,NaN
4,Unimig,U11005K,VIPER 135,WA INDUSTRIAL SUPPLIES EBAY,https://www.ebay.com.au/itm/186207493192?epid=...,NaN,NaN,NaN
22,Unimig,U11006K,VIPER 165,ELECTROWELD EBAY,https://www.ebay.com.au/itm/285104945700?itmme...,NaN,NaN,NaN
24,Unimig,U11006K,VIPER 165,WA INDUSTRIAL SUPPLIES EBAY,https://www.ebay.com.au/itm/186209047788?itmme...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1090,KUM390S/D,U11119,EVOLVE MULTI 400,HAMPDON EBAY,NaN,NaN,NaN,NaN
1093,KUM390S/D,U11119,EVOLVE MULTI 400,ELECTROWELD EBAY,https://www.ebay.com.au/itm/276821565411?_skw=...,NaN,NaN,NaN
1095,KUM390S/D,U11119,EVOLVE MULTI 400,WA INDUSTRIAL SUPPLIES EBAY,https://www.ebay.com.au/itm/186894851044?_skw=...,,https://www.ebay.com.au/itm/186906038061?_skw=...,(with wire feeder)
1103,KUM390S/D,U11119,EVOLVE MULTI 400,NATIONAL WELDING EBAY,NaN,NaN,NaN,NaN


In [12]:
#df_ebay['Shop Name'].unique()
df_ebay[(df_ebay['PRODUCT SKU'] == 'U11005K') & (df_ebay['PRODUCT NAME'] == 'VIPER 135') & (df_ebay['Shop Name'] == 'ELECTROWELD EBAY')].head()

,BRAND,PRODUCT SKU,PRODUCT NAME,Shop Name,PRODUCT LINK,Note,BUNDLE LINK,Comment
1,Unimig,U11005K,VIPER 135,ELECTROWELD EBAY,https://www.ebay.com.au/itm/275880137475?itmme...,NaN,NaN,NaN


In [13]:
#Test the get_price_original() with one sample record from ebay dataframe
df_ebay['Price_EBay']= df_ebay[(df_ebay['PRODUCT SKU'] == 'U11005K') & (df_ebay['PRODUCT NAME'] == 'VIPER 135') & (df_ebay['Shop Name'] == 'ELECTROWELD EBAY')]['PRODUCT LINK'].apply(get_price_original)

df_ebay[(df_ebay['PRODUCT SKU'] == 'U11005K') & (df_ebay['PRODUCT NAME'] == 'VIPER 135') & (df_ebay['Shop Name'] == 'ELECTROWELD EBAY')]

/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_45146/582628490.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ebay['Price_EBay']= df_ebay[(df_ebay['PRODUCT SKU'] == 'U11005K') & (df_ebay['PRODUCT NAME'] == 'VIPER 135') & (df_ebay['Shop Name'] == 'ELECTROWELD EBAY')]['PRODUCT LINK'].apply(get_price_original)


,BRAND,PRODUCT SKU,PRODUCT NAME,Shop Name,PRODUCT LINK,Note,BUNDLE LINK,Comment,Price_EBay
1,Unimig,U11005K,VIPER 135,ELECTROWELD EBAY,https://www.ebay.com.au/itm/275880137475?itmme...,NaN,NaN,NaN,AU $378.95


In [17]:
# Apply the get_price_original() on entire ebad df for fetching price for all applicable url rows
df_ebay['Price_EBay']= df_ebay['PRODUCT LINK'].apply(get_price_original)
df_ebay['Price_Bundle_EBay']= df_ebay['BUNDLE LINK'].apply(get_price_original)

/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_45146/1907644832.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ebay['Price_EBay']= df_ebay['PRODUCT LINK'].apply(get_price_original)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_45146/1907644832.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ebay['Price_Bundle_EBay']= df_ebay['BUNDLE LINK'].apply(get_price_original)


In [18]:
df_ebay

,BRAND,PRODUCT SKU,PRODUCT NAME,Shop Name,PRODUCT LINK,Note,BUNDLE LINK,Comment,Price_EBay,Price_Bundle_EBay
1,Unimig,U11005K,VIPER 135,ELECTROWELD EBAY,https://www.ebay.com.au/itm/275880137475?itmme...,NaN,NaN,NaN,AU $378.95,NaN
2,Unimig,U11005K,VIPER 135,HAMPDON EBAY,https://www.ebay.com.au/sch/i.html?_from=R40&_...,NaN,NaN,NaN,,NaN
4,Unimig,U11005K,VIPER 135,WA INDUSTRIAL SUPPLIES EBAY,https://www.ebay.com.au/itm/186207493192?epid=...,NaN,NaN,NaN,AU $399.00 each,NaN
22,Unimig,U11006K,VIPER 165,ELECTROWELD EBAY,https://www.ebay.com.au/itm/285104945700?itmme...,NaN,NaN,NaN,AU $646.95,NaN
24,Unimig,U11006K,VIPER 165,WA INDUSTRIAL SUPPLIES EBAY,https://www.ebay.com.au/itm/186209047788?itmme...,NaN,NaN,NaN,AU $729.00,NaN
...,...,...,...,...,...,...,...,...,...,...
1090,KUM390S/D,U11119,EVOLVE MULTI 400,HAMPDON EBAY,NaN,NaN,NaN,NaN,NaN,NaN
1093,KUM390S/D,U11119,EVOLVE MULTI 400,ELECTROWELD EBAY,https://www.ebay.com.au/itm/276821565411?_skw=...,NaN,NaN,NaN,"AU $9,499.00",NaN
1095,KUM390S/D,U11119,EVOLVE MULTI 400,WA INDUSTRIAL SUPPLIES EBAY,https://www.ebay.com.au/itm/186894851044?_skw=...,,https://www.ebay.com.au/itm/186906038061?_skw=...,(with wire feeder),"AU $9,499.00","AU $12,499.00"
1103,KUM390S/D,U11119,EVOLVE MULTI 400,NATIONAL WELDING EBAY,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_ebay.to_csv('Pricing_Ebay_append.csv',index=False)